## Category Imputation

In [1]:
import pandas as pd
import numpy as np
import arabic_reshaper as arabic_reshaper
from bidi.algorithm import get_display
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
import eli5
import csv

In [2]:
# read dataWithCleanٍSynopses dataSet
df_1= pd.read_csv('C:/Users/DELL/Documents/GitHub/Egyptian-Movies-Data-Analysis/Data/MoviesWithSynopsisAndCategory.csv')
df_2= pd.read_csv('C:/Users/DELL/Documents/GitHub/Egyptian-Movies-Data-Analysis/Data/MoviesWithoutSynopsisOrCategory.csv')
arr = [df_1,df_2]
df_movies = pd.concat(arr)
df_movies=df_movies.reset_index(drop=True)

In [3]:
df_movies.drop('بطولة ',axis='columns', inplace=True)

In [4]:
df_movies

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تمثيل,تأليف,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,بطولة اولي
0,قلب المرأة,1940,رومانسي,87,دار حدث فيلم خيري فتاه ثري مخطوب ابناء عموم تف...,"سليمان نجيب,أمينة رزق,دولت أبيض,عقيلة راتب,سلو...",توجو مزراحي (أحمد المشرقي),NaN,عبدالحليم نصر,NaN,NaN,NaN,NaN,توجو مزراحي (أحمد المشرقي),منتخبات بهنا فيلم,سليمان نجيب
1,الورشة,1940,دراما,105,سافر سطا عبدالرحمن صاحب ورشه ميكانزم زميل رحله...,"عزيزة أمير,محمود ذو الفقار,أنور وجدي,نجمة إبرا...","عزيزة أمير,محمود ذو الفقار",إيزيس فيلم,"أرام ماراليان,فيرى فاركاش (فرانسوا فاركاش)",NaN,محمد كامل,NaN,عبدالحميد عبدالرحمن,إستيفان روستي,منتخبات بهنا فيلم,عزيزة أمير
2,حياة الظلام,1940,دراما,94,عاءله متوسط حال مكون زوج زوج ٱبن حصل ٱبن اجازه...,"ميمي شكيب,محسن سرحان,روحية خالد,فردوس محمد,علي...","محمود كامل حسن,أحمد بدرخان",محمد جمال الدين رفعت,محمد عبدالعظيم,جمال مدكور,"علي عابد,روبرت شارفنبرج",NaN,"بيرم التونسي,عبدالحميد عبدالرحمن,محمد الكحلاوي...","أحمد بدرخان,فريد الجندي",NaN,ميمي شكيب
3,الباشمقاول,1940,دراما,85,دار حدث فيلم محامي ممتاز هوه مغازله نساء رغم ز...,"فوزي الجزايرلي,ميمي شكيب,إحسان الجزايرلي,زوزو ...","بديع خيري,توجو مزراحي (أحمد المشرقي)","توجو مزراحي (أحمد المشرقي),عبدالحميد زكي","محمود نصر,عبدالحليم نصر",NaN,NaN,NaN,فريد غصن,"توجو مزراحي (أحمد المشرقي),إبراهيم حلمي",منتخبات بهنا فيلم,فوزي الجزايرلي
4,يوم سعيد,1940,رومانسي,125,محمد كمال شاب بسيط هوه موسيقي غناء عطف صاحب بي...,"محمد عبدالوهاب,علوية جميل,فاطمة محجوب,فاتن حما...","محمد كريم,عبدالوارث عسر,عباس علام",أفلام عبدالوهاب,"محمد عبدالعظيم,جورج بنوا,بريما فيرا,مصطفى حسن",محمد كريم,حسين رستم,NaN,"محمد عبدالوهاب,أحمد رامي,عزت الهجين,بشارة الخو...",محمد كريم,NaN,محمد عبدالوهاب
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,اشانتي,1979,NaN,-,ملخص,عمر الشريف,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,عمر الشريف
2365,الرقص على أنغام البارود,1979,كوميدي,65,ملخص,"نجوى فؤاد,يونس شلبي,وحيد سيف,هدى سلطان,سهير زك...",فتحي عبدالستار,حسن موافى,NaN,NaN,NaN,NaN,NaN,"فتحي عبدالستار,عاطف شعلان",NaN,نجوى فؤاد
2366,لعنة الزمن,1979,NaN,105,ٱستغني عطيه صديق سطا حسن عامل مطبع ملك تصدي سط...,"فريد شوقي,عادل أدهم,بوسي,صلاح السعدني,خالد زكي...","آرثر ميلر,فيصل ندا","أفلام جمال التابعي,غطاس قلتة,جمال التابعي","علي خير الله,عادل أسعد","صلاح عبدالرازق,ليلى فهمي,حسين محمود,صلاح خليل",NaN,NaN,فؤاد الظاهري,"أحمد السبعاوي,سمير حافظ,كمال الحمصاني",الشركة اللبنانية للتجارة والاستثمار السينمائي ...,NaN
2367,لا يا أمي,1979,NaN,100,احمد مهندس شركه مقاول ملك ابراهيم وصل مركز مهم...,"فريد شوقي,مديحة كامل,مجدي وهبة,عماد حمدي,سيد ا...","ناصر حسين,محمد عثمان","الفيلم الماسى للإنتاج السينمائى,صلاح عمران,طلع...","عبدالمنعم بهنسي,غنيم بهنسي,محمد زغلول","فتحي داود,ليلى السايس,محمد ابراهيم",محمد إسماعيل,NaN,أحمد أبو زيد,"ناصر حسين,ملاك أندراوس,سيد فضل","سليمان فيلم,الشركة اللبنانية للتجارة والاستثما...",NaN


In [5]:
df_movies.replace('ملخص', np.nan, inplace=True)
df_movies.replace('-', np.nan, inplace=True)
df_movies["مدة الفيلم (دقيقة)"] = df_movies["مدة الفيلم (دقيقة)"].astype(float)

In [6]:
#check null count
print(df_movies.isnull().sum())

اسم الفيلم               0
تاريخ العرض              0
تصنيف الفيلم          1150
مدة الفيلم (دقيقة)     574
ملخص                   300
تمثيل                  114
تأليف                  284
إنتاج                  557
تصوير                  470
مونتاج                 638
ديكور                  910
ملابس                 2093
موسيقى                 980
إخراج                  193
توزيع                 1264
بطولة اولي             117
dtype: int64


In [7]:
#split our data into 3 dataSets
df_synoposis_And_category=df_movies[(df_movies['ملخص'].isnull()==False)&(df_movies['تصنيف الفيلم'].isnull()==False)]
df_synoposis_And_category=df_synoposis_And_category.reset_index(drop=True)

df_synoposis_And_NoCategory=df_movies[(df_movies['ملخص'].isnull()==False)&(df_movies['تصنيف الفيلم'].isnull()==True)]
df_synoposis_And_NoCategory=df_synoposis_And_NoCategory.reset_index(drop=True)

df_NoSynoposis=df_movies[(df_movies['ملخص'].isnull()==True)]
df_NoSynoposis=df_NoSynoposis.reset_index(drop=True)

## Classifier section

In [8]:
train= df_synoposis_And_category.copy()
test= df_synoposis_And_NoCategory.copy()

In [9]:
text_transformer = TfidfVectorizer( ngram_range=(1, 2),  max_features=150000)

In [10]:
%%time
X_train_text = text_transformer.fit_transform(train['ملخص'])
X_test_text = text_transformer.transform(test['ملخص'])

Wall time: 443 ms


In [11]:
X_train_text.shape,X_test_text.shape

((1183, 59595), (886, 59595))

In [12]:
logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)

In [13]:
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=17)

In [14]:
%%time
cv_results = cross_val_score(logit, X_train_text, train['تصنيف الفيلم'], cv=skf, scoring='f1_micro')

c:\python39\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"


Wall time: 33.5 s


In [15]:
cv_results, cv_results.mean()

(array([0.46790541, 0.47038917]), 0.4691472881510953)

In [16]:
%%time
logit.fit(X_train_text, train['تصنيف الفيلم'])

Wall time: 17.2 s


LogisticRegression(C=50.0, multi_class='multinomial', n_jobs=4, random_state=17)

In [17]:
#%%time
#eli5.show_weights(estimator=logit, 
#                  feature_names= list(text_transformer.get_feature_names()),
#                 top=(50, 5))

In [18]:
test_preds = logit.predict(X_test_text)
pd.DataFrame(test_preds, columns=['تصنيف'])

,تصنيف
0,دراما
1,دراما
2,دراما
3,دراما
4,دراما
...,...
881,رومانسي
882,دراما
883,دراما
884,دراما


In [19]:
# impute the missing values for تصنيف الفيلم based on classifier results
df_synoposis_And_NoCategory['تصنيف الفيلم']=test_preds

In [20]:
#merge the 3 data frames into One
frames = [df_synoposis_And_category,df_synoposis_And_NoCategory, df_NoSynoposis]
df_movies_updated = pd.concat(frames)
df_movies_updated=df_movies_updated.reset_index(drop=True)

In [21]:
print(df_movies_updated.isnull().sum())

اسم الفيلم               0
تاريخ العرض              0
تصنيف الفيلم           264
مدة الفيلم (دقيقة)     574
ملخص                   300
تمثيل                  114
تأليف                  284
إنتاج                  557
تصوير                  470
مونتاج                 638
ديكور                  910
ملابس                 2093
موسيقى                 980
إخراج                  193
توزيع                 1264
بطولة اولي             117
dtype: int64


In [22]:
#Do additinal mode imputation for the remainig null valus
df_movies_updated["تصنيف الفيلم"].fillna(df_movies_updated["تصنيف الفيلم"].mode()[0], inplace=True)
print(df_movies_updated.isnull().sum())

اسم الفيلم               0
تاريخ العرض              0
تصنيف الفيلم             0
مدة الفيلم (دقيقة)     574
ملخص                   300
تمثيل                  114
تأليف                  284
إنتاج                  557
تصوير                  470
مونتاج                 638
ديكور                  910
ملابس                 2093
موسيقى                 980
إخراج                  193
توزيع                 1264
بطولة اولي             117
dtype: int64


## Duration Imputation

In [23]:
df_movies_updated['تصنيف الفيلم'].describe()

count      2369
unique       20
top       دراما
freq       1581
Name: تصنيف الفيلم, dtype: object

In [24]:
print(df_movies_updated.isnull().sum())

اسم الفيلم               0
تاريخ العرض              0
تصنيف الفيلم             0
مدة الفيلم (دقيقة)     574
ملخص                   300
تمثيل                  114
تأليف                  284
إنتاج                  557
تصوير                  470
مونتاج                 638
ديكور                  910
ملابس                 2093
موسيقى                 980
إخراج                  193
توزيع                 1264
بطولة اولي             117
dtype: int64


In [25]:
# method defiend for multivarient imputation
def impute_numerical(df,categorical_column, numerical_column):
    print(categorical_column)
    frames_func = []
    for i in list(set(df[categorical_column])):
        df_category = df[df[categorical_column]== i]
        if len(df_category) > 1:    
            df_category[numerical_column].fillna(df_category[numerical_column].mean(),inplace = True)        
        else:
            df_category[numerical_column].fillna(df[numerical_column].mean(),inplace = True)
        frames_func.append(df_category)    
        final_df = pd.concat(frames_func)
    return final_df

In [26]:
#impute duration for all null values based on the category
df_movies_final  = impute_numerical(df_movies_updated,'تصنيف الفيلم','مدة الفيلم (دقيقة)')
df_movies_final=df_movies_final.reset_index(drop=True)


تصنيف الفيلم


c:\python39\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [27]:
print(df_movies_final.isnull().sum())

اسم الفيلم               0
تاريخ العرض              0
تصنيف الفيلم             0
مدة الفيلم (دقيقة)       0
ملخص                   300
تمثيل                  114
تأليف                  284
إنتاج                  557
تصوير                  470
مونتاج                 638
ديكور                  910
ملابس                 2093
موسيقى                 980
إخراج                  193
توزيع                 1264
بطولة اولي             117
dtype: int64


In [28]:
df_movies_final.head(3)

,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تمثيل,تأليف,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,بطولة اولي
0,رابحة,1945,تاريخي,105.0,رابح فتاه بدوي قبيل متعصب ٱلتقي صدفه شاب مدينه...,"كوكا,بدر لاما,سراج منير,عباس فارس,بشارة واكيم,...","بيرم التونسي,محمود تيمور,نيازي مصطفى",NaN,"محمد عبدالعظيم,مصطفى حسن",جلال مصطفى,ولي الدين سامح,NaN,"عبدالحميد عبدالرحمن,محمد الكحلاوي,آمال حسين",نيازي مصطفى,NaN,كوكا
1,فتح مصر,1948,تاريخي,110.0,تناول فيلم حقب تاريخ شعب مصري يكابد شتي نوع عذ...,"لولا صدقي,أحمد البيه,منسى فهمي,رفيعة الشال,أحم...","جليل البنداري,محمد صبيح,فؤاد الجزايرلي",أفلام النسر,أوهان,جلال مصطفى,روبرت شارفنبرج,NaN,NaN,فؤاد الجزايرلي,ستوديو مصر,لولا صدقي
2,ظهور الإسلام (الوعد الحق),1951,تاريخي,95.0,تناول عمل حياه جزيره عربي اظهر اسلام حياه عبار...,"كوكا,عماد حمدي,عباس فارس,سراج منير,زكي إبراهيم...","إبراهيم عز الدين,طه حسين",أفلام المصري (إبراهيم عز الدين عزقلاني),"والتر هاجوب,والترها جوب,حسن البحيري,محمد عز العرب","حسن حلمي (حسنوف),كمال الشيخ","ستوديو مصر,نجيب خوري,أنطوان بوليزويس",NaN,NaN,"إبراهيم عز الدين,حمادة عبدالوهاب,شريف حمودة",NaN,كوكا


In [29]:
df_movies_final.to_csv('df_imputed&cleaned.csv')